In [1]:
from pulser import Pulse, Sequence, Register
from pulser.devices import Chadoq2
from simulation import Simulation

from pulser.waveforms import BlackmanWaveform,ConstantWaveform

simulation module...


In [2]:
import qutip
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def CZ(initial,distances=np.linspace(7,20,20),plots=False):
    """
    Implements the CZ gate pulse sequence. A range of distances between the atoms
    is automatically given but one can give specific values as parameters.
    Notice that the time steps are defined automatically
    """
    
    duration = 1600
    pi_pulse = Pulse(BlackmanWaveform(duration,np.pi),ConstantWaveform(duration,0),0)
    twopi_pulse = Pulse(BlackmanWaveform(duration,2*np.pi),ConstantWaveform(duration,0),0)
    
    evolution_r =[]
    evolution_i =[]
    
    for R in distances:
        # Atom Register
        L = 2
        reg = Register.rectangle(1, L, spacing=R, prefix='atom')
        
        # Define Device 
        device = Chadoq2(reg) 
        
        # Define sequence and Set channels
        seq = Sequence(device)
        seq.declare_channel('rydberg', 'rydberg_local')
        seq.declare_channel('raman', 'raman_local') # Declared but unused so that a three-state basis is needed
        
        seq.target('atom1','raman') #control
        seq.target('atom0','rydberg') #control
        seq.add(pi_pulse,'rydberg')
        seq.target('atom1','rydberg') #target
        seq.add(twopi_pulse,'rydberg')
        seq.target('atom0','rydberg') #control again
        seq.add(pi_pulse,'rydberg')
        #seq.draw()
        
        simul = Simulation(seq)
        #No observable: We want output state
        simul.run(plot=False)
        state = simul.output

        scalar_prod = np.vdot(initial.data.toarray(),state.data.toarray())
        evolution_r.append(np.real(scalar_prod))
        #evolution_i.append(np.imag(scalar_prod))
    
    if initial == qutip.tensor([qutip.basis(3,1),qutip.basis(3,1)]): names = 'hh' 
    if initial == qutip.tensor([qutip.basis(3,1),qutip.basis(3,2)]): names = 'hg' 
    if initial == qutip.tensor([qutip.basis(3,2),qutip.basis(3,1)]): names = 'gh' 
    if initial == qutip.tensor([qutip.basis(3,2),qutip.basis(3,2)]): names = 'gg' 
    
    if plots:
        plt.plot(distances,evolution_r, label=f"Re <{names}|psi_out>")
        #plt.plot(distances,evolution_i,label=f"Im <{names}|psi_out>")
        plt.title('Overlap with the initial state')
        plt.legend()
    
    return scalar_prod

In [ ]:
states = [qutip.tensor([qutip.basis(3,1),qutip.basis(3,1)]), 
          qutip.tensor([qutip.basis(3,1),qutip.basis(3,2)]),
          qutip.tensor([qutip.basis(3,2),qutip.basis(3,1)]),
          qutip.tensor([qutip.basis(3,2),qutip.basis(3,2)])
         ]

for initial in states:
    CZ(initial, plots=True)